In [1]:
# Import Libraries

import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta, date
import h2o

In [2]:
# Create DF to Fill Empty Dates - Não Precisa

def set_hour(date1):
  day1 = datetime(date1.year, date1.month, date1.day,0,0,0)
  return day1

dftime = pd.DataFrame(pd.period_range(start=datetime(2019,11,4,0,0,0), periods=24, freq='7D1H', name = 'Date1'))
dftime['date'] = dftime['Date1'].apply(lambda x: set_hour(x))
dftime = dftime.drop('Date1', 1)
dftime.set_index('date', inplace = True)

In [3]:
# Connection data to Database

PATH = '/Users/renato/Downloads/Crystalball/'
DATABASE_MODEL = 'modelsample.db'
MODEL_CSV = 'model.csv'
DATABASE_HISTORY = 'history.db'
DATABASE_CACHE = 'cache.db'
USER = 'user'
PASSWORD = 'password'
PORT = '5432'
HOST = '127.0.0.1'

In [4]:
# Default Data

CPF = '14566178854'

In [5]:
def get_data(cpf):
    try:
        connection = PATH + DATABASE_HISTORY
        sqliteConnection = sqlite3.connect(connection)
        cursor = sqliteConnection.cursor()
        sqlite_select_query = """SELECT * from history_purchase where cpf = """ + cpf
        cursor.execute(sqlite_select_query)
        records = cursor.fetchall()
        records = pd.DataFrame(records)
        records.columns = ['cpf', 'date1', 'prod_id', 'qtde']
        records['date'] =  pd.to_datetime(records['date1'],  format='%Y-%m-%d' )
        records = records.drop('date1', 1)

        cursor.close()
        
        return(records)

    except sqlite3.Error as error:
        print("Failed to read data from sqlite table", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()

In [40]:
# Feature Engineering

def feature_engineering(df):

    
    #Load Model
    localmodelo = PATH + "h2omodel.dat"
    model = h2o.load_model(localmodelo)
    
    
    # DataFrame for Loop
    dfloop = df.groupby(['prod_id'], as_index=False)['qtde'].count()
    dfloop

    data = pd.DataFrame()
    data['qtde'] = 1
    data['lag1'] = 2
    data['lag2'] = 3
    data['lag3'] = 4
    data['lag4'] = 5
    data['lag5'] = 5
    data['lag6'] = 7
    data['lag7'] = 8
    data['lag8'] = 9
                
    
    #Iteract for product
    for i, r in dfloop.iterrows():
        # Filtrar Item_id em um novo DF
        dffilter = df[df['prod_id'] == r['prod_id']]

        # merge data
        dffilter['date1'] = dffilter['date']
        dffilter.set_index('date', inplace = True)
        dfmerge = pd.concat([dffilter,dftime], axis=1)
        dfmerge['date1'] = dfmerge.index
        dfmerge['cpf'] = CPF
        dfmerge['prod_id'] = r['prod_id']
        dfmerge = dfmerge.fillna(0)

        
        #Colocar lags
        dfmerge['qtde'] = dfmerge['qtde']
        dfmerge['lag1'] = dfmerge['qtde']
        dfmerge['lag2'] = dfmerge.qtde.shift(1)
        dfmerge['lag3'] = dfmerge.qtde.shift(2)
        dfmerge['lag4'] = dfmerge.qtde.shift(3)
        dfmerge['lag5'] = dfmerge.qtde.shift(4)
        dfmerge['lag6'] = dfmerge.qtde.shift(5)
        dfmerge['lag7'] = dfmerge.qtde.shift(6)
        dfmerge['lag8'] = dfmerge.qtde.shift(7)
        dfmerge = dfmerge.dropna()

        
        #Drop Columns
        dfmerge = dfmerge.drop('cpf', 1)
        dfmerge = dfmerge.drop('prod_id', 1)
        dfmerge = dfmerge.drop('date1', 1)
        
        # Salvar previsão em DF
        dfmerge = [dfmerge.iloc[-1,:]]
        data = pd.DataFrame(dfmerge)
        print(data)
        h2o_df = h2o.H2OFrame(data)
        pred = model.predict(h2o_df)
        predict = pred.as_data_frame()
        
        print(predict)

In [41]:
h2o.init()


df = get_data (CPF)
engdata = feature_engineering(df)
#retorno = save_data(engdata)
#retorno

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,8 mins 52 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.1
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_renato_uzadot
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.999 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


/Volumes/Dados/Users/renato/anaconda3/envs/dev_phyton/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            qtde  lag1  lag2  lag3  lag4  lag5  lag6  lag7  lag8
2020-04-13   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0
Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
    predict
0  1.007617


/Volumes/Dados/Users/renato/anaconda3/envs/dev_phyton/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

            qtde  lag1  lag2  lag3  lag4  lag5  lag6  lag7  lag8
2020-04-13   1.0   1.0   2.0   3.0   4.0   5.0   6.0   7.0   8.0
Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
    predict
0  0.222349


/Volumes/Dados/Users/renato/anaconda3/envs/dev_phyton/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            qtde  lag1  lag2  lag3  lag4  lag5  lag6  lag7  lag8
2020-04-13   0.0   0.0   3.0   0.0   3.0   0.0   3.0   0.0   3.0
Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
    predict
0  1.976483


/Volumes/Dados/Users/renato/anaconda3/envs/dev_phyton/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            qtde  lag1  lag2  lag3  lag4  lag5  lag6  lag7  lag8
2020-04-13   0.0   0.0   0.0   4.0   0.0   0.0   4.0   0.0   4.0
Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
    predict
0 -0.732191
